In [ ]:
!pip install langchain_community
!pip install faiss--cpu
!pip install langchain_groq
!pip install wikipedia

LLM

In [ ]:
from langchain_groq import ChatGroq
llm=ChatGroq(api_key="",model="llama-3.3-70b-versatile")

In [ ]:
llm.invoke("hello")

AIMessage(content='Hello. How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 36, 'total_tokens': 46, 'completion_time': 0.036363636, 'prompt_time': 0.007817468, 'queue_time': 0.433037761, 'total_time': 0.044181104}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_1ae2705acb', 'finish_reason': 'stop', 'logprobs': None}, id='run-5591bd0c-6af2-4c0c-ba27-3d8c42ce1097-0', usage_metadata={'input_tokens': 36, 'output_tokens': 10, 'total_tokens': 46})

In [ ]:
#prompt
#tools

Tools

In [ ]:
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper=WikipediaAPIWrapper()
wiki_tool=WikipediaQueryRun(api_wrapper=api_wrapper)

In [ ]:
wiki_tool.invoke("Hitler")

'Page: Adolf Hitler\nSummary: Adolf Hitler (20 April 1889 – 30 April 1945) was an Austrian-born German politician who was the dictator of Nazi Germany from 1933 until his suicide in 1945. He rose to power as the leader of the Nazi Party, becoming the chancellor in 1933 and then taking the title of Führer und Reichskanzler in 1934. His invasion of Poland on 1 September 1939 marked the start of the Second World War. He was closely involved in military operations throughout the war and was central to the perpetration of the Holocaust: the genocide of about six million Jews and millions of other victims.\nHitler was born in Braunau am Inn in Austria-Hungary and was raised near Linz. He lived in Vienna in the first decade of the 1900s before moving to Germany in 1913. He was decorated during his service in the German Army in World War I, receiving the Iron Cross. In 1919, he joined the German Workers\' Party (DAP), the precursor of the Nazi Party, and in 1921 was appointed leader of the Naz

VANILLA CHAIN

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import MessagesPlaceholder
prompt1=ChatPromptTemplate.from_messages([
    ("system","Play the historical figure {historical_figure} which the user inputs and then answer user questions"),
    ("user","{user_input}"),
])

In [ ]:
chain=prompt1|llm

In [ ]:
chain.invoke({"historical_figure":"Aristotle","user_input":"hello"})

AIMessage(content='Greetings to you, my young friend. I am Aristotle, a philosopher and polymath from ancient Greece. I was born in Stagira, a small town in Macedonia, in the year 384 BCE. I have spent my life studying and teaching various subjects, including philosophy, science, and ethics. I have had the privilege of being a student of the great Plato and a tutor to the legendary Alexander the Great.\n\nI must say, I am intrigued by the opportunity to converse with someone from a future era. I am eager to learn about the advancements and discoveries that have taken place since my time. And, of course, I am happy to share my own knowledge and insights with you.\n\nWhat would you like to discuss, my friend? The nature of reality, the principles of ethics, or perhaps the movements of the celestial bodies?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 169, 'prompt_tokens': 50, 'total_tokens': 219, 'completion_time': 0.614545455, 'prompt_time': 0.01020964

AGENTS

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import MessagesPlaceholder
prompt2 = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful and informative AI assistant. You are playing the role of the historical figure specified by the user. "
              "Do not break character.  If you do not know the answer to a question, you may use tools to find the answer. "
              "You have access to a Wikipedia tool to research information about the historical figure."),
    ("user", "{user_input},{historical_figure}"),
    (MessagesPlaceholder(variable_name="chat_history")),
    (MessagesPlaceholder(variable_name="agent_scratchpad"))
])

In [ ]:
#solving the same problem statement using
from langchain.agents import create_openai_functions_agent, AgentExecutor
agent=create_openai_functions_agent(llm, tools=[], prompt=prompt2)
agent_exec=AgentExecutor(agent=agent,prompt=prompt2,tools=[],vrbose=True)

In [ ]:
class AgentExecutions:
    def __init__(self, query, historical_figure):
        self.query = query
        self.historical_figure = historical_figure
        self.chat_history = []  # Initialize chat history as an instance attribute
        #basically this is a list which gets appended and updated

    def run(self):
        self.chat_history.append(self.query)
        self.chat_history.append(self.historical_figure)
        # Ensure `agent_exec` is defined elsewhere in your code
        result = agent_exec.invoke({
            "historical_figure": self.historical_figure,
            "user_input": self.query,
            "chat_history": self.chat_history
        })
        return result  # Return result for further usage



In [ ]:
a1=AgentExecutions("when were you born","tupac")


In [ ]:
a1.run()

{'historical_figure': 'tupac',
 'user_input': 'when were you born',
 'chat_history': ['when were you born', 'tupac'],
 'output': "What's good fam? I was born Lesane Parish Crooks on June 16, 1971, in East Harlem, New York City. But you know me as Tupac Amaru Shakur, the voice of the streets, the poet, the rapper, and the revolutionary. Word."}

In [ ]:
#solve the issue of chat history using classes and objects
#that way we will get a sessions chat history

SOLVING THE SAME PROBLEM USING A MULTIAGENTIC SYSTEM

In [ ]:
!pip install crewai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.9/211.9 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.5/605.5 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 89.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 4.5 MB/s eta 0:

In [ ]:
from google.colab import userdata
OPEN_AI_KEY=userdata.get('OPEN_AI_KEY')

import os
os.environ['OPENAI_API_KEY']=OPEN_AI_KEY

In [ ]:
from crewai import Agent

researcher_agent=Agent(
    role="AI researcher",
    goal="Conduct a thorough research on the way the historical figure{historical_figure} input by the user talks. Deep dive into the nuances and specifics in which the speech of the historical figure ",
    backstory="""You are a speech expert specializing in mimicking historical figures """,
    verbose=True,
    allow_delegation=True
)

In [ ]:
responder_agent=Agent(
    role="craft responses",
    goal="craft responses to the user input {user_input} posing as the historical figure {historical_figure}",
    backstory=""" You are an expert in crafting user responses""",
    verbose=True,
    allow_delegation=True
)

In [ ]:
from crewai import Task

In [ ]:
task1=Task(
    description="Study the way historical figures talk and study the nuances in their speeches",
    expected_output="a small reposrt covering all he nuances in their manner of speaking",
    agent=researcher_agent
)

In [ ]:
task2=Task(
    description="study the provided report and draft a response to the user query",
    expected_output="a small response to the user query in the same manner the historical figure would respond",
    agent=researcher_agent
)

In [ ]:
#the process is like the agent executor
from crewai import Crew, Process
crew=Crew(
    agents=[researcher_agent,responder_agent],
    tasks=[task1,task2],
    verbose=True,
    process=Process.sequential,
    full_output=True
)

In [ ]:
crew.kickoff({"historical_figure":"elvis","user_input":"when were you born"})

# Agent: AI researcher
## Task: Study the way historical figures talk and study the nuances in their speeches
 

I encountered an error while trying to use the tool. This was the error: Arguments validation failed: 2 validation errors for DelegateWorkToolSchema
task
  Input should be a valid string [type=string_type, input_value={'description': "Conduct ...eaking.", 'type': 'str'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/string_type
context
  Input should be a valid string [type=string_type, input_value={'description': 'Focus on...uances.', 'type': 'str'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/string_type.
 Tool Delegate work to coworker accepts these inputs: Tool Name: Delegate work to coworker
Tool Arguments: {'task': {'description': 'The task to delegate', 'type': 'str'}, 'context': {'description': 'The context for the task', 'type': 'str'}, 'coworker': {'description': 'The role/name of the

CrewOutput(raw='"Well, honey, let me tell ya, I was born on January 8, 1935, in a little place called Tupelo, Mississippi. Thank ya, thank ya very much for askin\'! That day was a real shakin\' start to my rock \'n\' roll life, makin’ waves and stirrin’ hearts ever since. Now ain\'t that somethin\'? Uh-huh!"', pydantic=None, json_dict=None, tasks_output=[TaskOutput(description='Study the way historical figures talk and study the nuances in their speeches', name=None, expected_output='a small reposrt covering all he nuances in their manner of speaking', summary='Study the way historical figures talk and study the nuances...', raw='**Report on Elvis Presley\'s Speech Patterns**\n\nElvis Presley, known as the "King of Rock and Roll," not only revolutionized music but also left a significant mark on American culture through his distinct speech patterns. The nuances of his way of speaking are deeply influenced by his Southern upbringing, his musical career, and his unique personality traits